<a href="https://colab.research.google.com/github/vkjadon/openai/blob/main/04oai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We can load the api key from `.env` file by importing `load_dotenv` method of `dotenv` package. We can install `dotenv` package as

> `pip install python-dotenv`

> `from dotenv import load_dotenv, find_dotenv`

`find_dotenv()` : This function searches up the directory tree from your current working directory until it locates the .env file.

`load_dotenv()` : This function reads the variables from the found .env file and loads them into your Python process's environment variables (specifically, os.environ).

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

In case you are running the code on Google Colab, add your "OPENAI_API_KEY" key in the `Secrets` of the colab environment and call instantiate the `OpenAI` object as below

In [28]:
!pip install -q faiss-cpu pypdf tiktoken

In [29]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [30]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving 08linearRegressionMF.ipynb - Colab.pdf to 08linearRegressionMF.ipynb - Colab.pdf


In [33]:
from pypdf import PdfReader

reader = PdfReader(pdf_path)

full_text = ""
scanned_pages = []

for i, page in enumerate(reader.pages):
    page_text = page.extract_text()

    if page_text:
        full_text += page_text + "\n"
    else:
        scanned_pages.append(i)

print("Preview:\n", full_text[:1000])
print("Scanned pages:", scanned_pages)

Preview:
 Linear Regression (Multi Features and m Training Examples)
We will use  to denote the feature vector and  to denote output variable for 
training example.
Where,
 : training examples (  to )
 : Number of features
Let us write  for the prediction from the linear model.
Feature vector for  training example: 
Feature vector of the problem dataset:
Parameter vector : 
𝐱(𝐢) 𝐲(𝐢) 𝐢𝐭𝐡
ℎ (𝑥)= + +.....+ +𝑏𝑤1𝑥(𝑖)
1 𝑤2𝑥(𝑖)
2 𝑤𝑛𝑥𝑥(𝑖)𝑛𝑥
𝑚 𝑖 =1 𝑚
𝑛𝑥
𝑦̂ 
= + +.....+ +𝑏𝑦̂ (𝑖) 𝑤1𝑥(𝑖)
1 𝑤2𝑥(𝑖)
2 𝑤𝑛𝑥𝑥(𝑖)𝑛𝑥
𝑖𝑡ℎ =𝐱(𝑖)
⎛
⎝
⎜⎜⎜⎜⎜
𝑥(𝑖)
1
𝑥(𝑖)
2
⋮ 
𝑥(𝑖)𝑛𝑥
⎞
⎠
⎟⎟⎟⎟⎟
𝐗=( )𝐱(1) 𝐱(2) … 𝐱(𝑚)
𝐗=
⎛
⎝
⎜⎜⎜⎜⎜
𝑥(1)
1
𝑥(1)
2
⋮ 
𝑥(1)𝑛𝑥
𝑥(2)
1
𝑥(2)
2
⋮ 
𝑥(1)𝑛𝑥
…
…
…
…
𝑥(𝑚)
1
𝑥(𝑚)
2
⋮ 
𝑥(𝑚)𝑛𝑥
⎞
⎠
⎟⎟⎟⎟⎟
𝐰= ,𝑏
⎛
⎝
⎜⎜⎜⎜
𝑤1
𝑤2
⋮ 
𝑤𝑛𝑥
⎞
⎠
⎟⎟⎟⎟
Size Bedrooms Price (L)
100 1 20
150 2 28
200 3 39
250 4 51
500 4 80
Example Data
12/15/25, 3:13 PM 08linearRegressionMF.ipynb - Colab
https://colab.research.google.com/github/vkjadon/DNN/blob/master/08linearRegressionMF.ipynb#printMode=true 1/8
Features for  training example:
F

In [32]:
def chunk_text(text, chunk_size=800, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

In [7]:
chunks = chunk_text(text)
print("Total chunks:", len(chunks))

Total chunks: 315


In [8]:
import numpy as np

In [ ]:
def get_embeddings(texts):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return np.array([e.embedding for e in response.data])

In [10]:
chunk_embeddings = get_embeddings(chunks)
print(chunk_embeddings.shape)

(315, 1536)


FAISS (Facebook AI Similarity Search) is an open-source library for efficient similarity search and clustering of dense, high-dimensional vectors. It is used to quickly find items in massive datasets that are "similar" to a given query item, solving performance bottlenecks that traditional databases face when dealing with unstructured data like images or text embeddings.

In [11]:
import faiss

dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)

print("FAISS index size:", index.ntotal)

FAISS index size: 315


In [12]:
def retrieve_chunks(query, k=3):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

In [13]:
def build_context(query):
    retrieved = retrieve_chunks(query)
    return "\n\n".join(retrieved)

In [14]:
def ask_rag(question):
  context = build_context(question)

  response = client.responses.create(
      model="gpt-4.1-mini",
      input=[
        {
          "role": "system",
          "content": "You are a helpful teacher."
        },
        {
          "role": "developer",
          "content": (
                        "Answer ONLY using the provided context. "
                        "If the answer is not present, say 'I do not know.'"
                    )
        },
        {
          "role": "user",
          "content": f"""
                      {context}

                      {question}
                      """
        }
        ]
      )
  return response.output_text


In [22]:
question = "What is the material for a gear."
answer = ask_rag(question)

print(answer)


The materials for gears mentioned in the provided context are:

- C14 (used for light duty gears)
- C45 (shafts, bolts, gears and spindles of machine tools)
- C50Mn1 (bolts, gear shafts)
- C55M75 (gears, sprockets, parts requiring moderate wear resistance)
- C40 (gears etc)
